In [1]:
import torch
from  torch.utils.data import TensorDataset, DataLoader


In [3]:
# X1=torch.load('/home/free4ky/projects/chest-diseases/data/preprocessed_train/train_data.pt')
Y1=torch.load('/home/free4ky/projects/chest-diseases/data/preprocessed_train_20/train_labels.pt')
Y2=torch.load('/home/free4ky/projects/chest-diseases/data/preprocessed_train_20/val_labels.pt')
Y3=torch.load('/home/free4ky/projects/chest-diseases/data/preprocessed_mosmed/test_labels.pt')
Y4=torch.load('/home/free4ky/projects/chest-diseases/data/preprocessed_mosmed/train_labels.pt')
Y5=torch.load('/home/free4ky/projects/chest-diseases/data/preprocessed_mosmed/val_labels.pt')

# X = torch.concat([X1,X2,X3],dim=0)
Y = torch.concat([Y1,Y2,Y3,Y4,Y5],dim=0)
print(Y.shape)

torch.Size([49012, 20])


In [5]:

ds = TensorDataset(
        Y
)
dl = DataLoader(
    ds,
    batch_size=1,
    shuffle=False
)


In [6]:
ds.tensors[0].shape

torch.Size([49012, 20])

In [7]:
def compute_pos_weights(labels: torch.Tensor) -> torch.Tensor:
    """
    Compute pos_weight values for BCEWithLogitsLoss given multilabel dataset.
    
    Args:
        labels (torch.Tensor): shape [num_samples, num_classes], binary (0/1).
    
    Returns:
        torch.Tensor: pos_weight for each class, shape [num_classes].
    """
    num_samples, num_classes = labels.shape
    
    # count positives and negatives per class
    pos_counts = labels.sum(dim=0)                  # [num_classes]
    neg_counts = num_samples - pos_counts           # [num_classes]
    
    # avoid division by zero (if class never appears)
    pos_counts = torch.clamp(pos_counts, min=1)
    
    pos_weight = neg_counts / pos_counts
    return pos_weight


In [8]:
labesl = ds.tensors[-1]
labesl.shape

torch.Size([49012, 20])

In [9]:
compute_pos_weights(labesl)

tensor([ 7.4242,  2.6639,  8.2336, 13.3646,  3.0758,  6.2600,  3.0108,  4.3729,
         2.9967,  1.2923,  1.8135,  2.8932,  7.5911, 12.4722,  8.8556,  4.8923,
         9.3576, 12.0873, 51.9287, 82.6382])

In [34]:
weights = torch.tensor([9.211362733, 2.384068466, 8.295479204, 32.8629776, 2.992233613,
                        6.064870808, 3.176470588, 4.187083754, 3.022222222, 1.216071737,
                        1.677849552, 3.152851834, 7.123261694, 18.16629381, 13.8480647,
                        6.335045662, 10.81701149, 13.40695067]).cuda()
weights.sum()

tensor(147.9402, device='cuda:0')

In [12]:
def compute_alt_pos_weights(labels: torch.Tensor) -> torch.Tensor:
    num_samples, _ = labels.shape
    pos_counts = labels.sum(dim=0)
    pos_counts = torch.clamp(pos_counts, min=1)
    return num_samples / pos_counts

In [15]:
weights  = compute_alt_pos_weights(labesl)
weights

tensor([ 8.4242,  3.6639,  9.2336, 14.3646,  4.0758,  7.2600,  4.0108,  5.3729,
         3.9967,  2.2923,  2.8135,  3.8932,  8.5911, 13.4722,  9.8556,  5.8923,
        10.3576, 13.0873, 52.9287, 83.6382])

In [44]:

given = torch.tensor([9.211362733, 2.384068466, 8.295479204, 32.8629776,
                      2.992233613, 6.064870808, 3.176470588, 4.187083754,
                      3.022222222, 1.216071737, 1.677849552, 3.152851834,
                      7.123261694, 18.16629381, 13.8480647, 6.335045662,
                      10.81701149, 13.40695067])

computed = (N / pos_counts)   # your result from code

ratios = given / computed
print(ratios)

tensor([1.1253, 0.6767, 0.9311, 2.3823, 0.7626, 0.8663, 0.8234, 0.8111, 0.7814,
        0.5511, 0.6220, 0.8431, 0.8631, 1.4085, 1.4702, 1.1234, 1.0911, 1.0670])


In [ ]:
def compute_all_pos_weights(labels: torch.Tensor):
    """
    Compute different variants of pos_weight for BCEWithLogitsLoss.
    
    Args:
        labels (torch.Tensor): shape [num_samples, num_classes], binary (0/1).
    
    Returns:
        dict: dictionary of {method_name: pos_weights_tensor}
    """
    num_samples, num_classes = labels.shape
    
    pos_counts = labels.sum(dim=0)                # positives per class
    neg_counts = num_samples - pos_counts         # negatives per class
    
    # avoid division by zero
    pos_counts = torch.clamp(pos_counts, min=1)
    neg_counts = torch.clamp(neg_counts, min=1)

    results = {}

    # 1. Classic: neg/pos
    results["neg/pos"] = neg_counts / pos_counts

    # 2. Total/pos
    results["N/pos"] = num_samples / pos_counts

    # 3. Log-scaling of N/pos
    results["log(N/pos)"] = torch.log1p(num_samples / pos_counts)

    # 4. Sqrt-scaling of N/pos
    results["sqrt(N/pos)"] = torch.sqrt(num_samples / pos_counts)

    # 5. Smoothing ( (neg+1)/(pos+1) )
    results["smoothed"] = (neg_counts + 1) / (pos_counts + 1)

    # 6. Per-class frequency normalization
    #   freq[c] = pos_counts[c] / N
    #   pos_weight[c] = mean(freq) / freq[c]
    freqs = pos_counts / num_samples
    mean_freq = freqs.mean()
    results["freq_norm"] = mean_freq / freqs

    return results



In [33]:


labels = ds.tensors[-1]
weights_dict = compute_all_pos_weights(labels)
for k, v in weights_dict.items():
    print(f"{k}: {v}")

neg/pos: tensor([ 7.1855,  2.5232,  7.9091, 12.7947,  2.9238,  6.0013,  2.8577,  4.1620,
         2.8675,  1.2067,  1.6975,  2.7396,  7.2528, 11.8977,  8.4191,  4.6394,
         8.9141, 11.5651])
N/pos: tensor([ 8.1855,  3.5232,  8.9091, 13.7947,  3.9238,  7.0013,  3.8577,  5.1620,
         3.8675,  2.2067,  2.6975,  3.7396,  8.2528, 12.8977,  9.4191,  5.6394,
         9.9141, 12.5651])
log(N/pos): tensor([2.2176, 1.5092, 2.2935, 2.6943, 1.5941, 2.0796, 1.5806, 1.8184, 1.5826,
        1.1652, 1.3077, 1.5559, 2.2249, 2.6317, 2.3436, 1.8930, 2.3901, 2.6075])
sqrt(N/pos): tensor([2.8610, 1.8770, 2.9848, 3.7141, 1.9809, 2.6460, 1.9641, 2.2720, 1.9666,
        1.4855, 1.6424, 1.9338, 2.8728, 3.5913, 3.0691, 2.3747, 3.1487, 3.5447])
smoothed: tensor([ 7.1844,  2.5231,  7.9079, 12.7914,  2.9236,  6.0006,  2.8576,  4.1617,
         2.8674,  1.2067,  1.6975,  2.7394,  7.2517, 11.8949,  8.4177,  4.6390,
         8.9125, 11.5625])
freq_norm: tensor([1.5706, 0.6760, 1.7095, 2.6469, 0.7529, 1.3434,